In [1]:
from twscrape import AccountsPool

In [3]:
pool = AccountsPool()
await pool.load_from_file(filepath="C:/Users/Admin/accounts.txt", line_format='username:password:email:email_password')

2024-01-15 19:52:49.456 | WARNING  | twscrape.accounts_pool:add_account:76 - Account CaoduyV already exists


In [4]:
await pool.login_all()

{'total': 0, 'success': 0, 'failed': 0}

In [5]:
import asyncio
from twscrape import API, gather
from twscrape.logger import set_log_level
import numpy as np
import pandas as pd
import tqdm

COLUMNS = ['conversationId', 'coordinates', 'date', 'dict', 'hashtags', 'id', 'id_str', 'inReplyToTweetId', 'inReplyToUser', 'json', 'lang', 'likeCount', 'links', 'media', 'mentionedUsers', 'parse', 'place', 'quoteCount', 'quotedTweet', 'rawContent', 'replyCount', 'retweetCount', 'retweetedTweet', 'source', 'sourceLabel', 'sourceUrl', 'url', 'user', 'viewCount']

async def get_data(args):
    df = pd.DataFrame()
    api = API(pool=pool)
    keywork, since, until = args
    
    q = f"{keywork} since:{since} until:{until}"
    async for tweet in api.search(q, limit=50):
#         print("DEBUG df_t",{c:getattr(tweet, c) for c in COLUMNS if getattr(tweet, c)})
        data = {}
        for c in COLUMNS:
            x = getattr(tweet, c)
            if isinstance(x, list):
                data[c]= [x]
            else:
                data[c] = x
        df_t = pd.DataFrame(data)
        
        df = pd.concat([df, df_t])
        time.sleep(np.random.randint(3))
    return df

In [7]:
from datetime import datetime, timedelta

start_date = '2020-01-01'
start_date = datetime.strptime(start_date, '%Y-%m-%d')
end_date = '2022-01-01'
end_date = datetime.strptime(end_date, '%Y-%m-%d')

d = start_date
end_points = []
while d < end_date:
    next_d = d + timedelta(days=2)
    end_points.append(('bitcoin', d.strftime('%Y-%m-%d'), next_d.strftime('%Y-%m-%d')))
    d = d + timedelta(days=2)

In [11]:
import time

for end_point in tqdm.tqdm(end_points[1:]):
# for end_point in tqdm.tqdm(end_points):
    df_t = await get_data(end_point)
    df = pd.concat([df,df_t])
    time.sleep(np.random.randint(10))

 28%|█████████████████████▋                                                      | 104/365 [2:08:34<5:43:45, 79.02s/it]2024-01-15 22:05:59.679 | ERROR    | twscrape.models:_write_dump:418 - Failed to parse response of tweet, writing dump to /tmp/twscrape/twscrape_parse_error_2024-01-15_15-05-59_lkrvq.txt
2024-01-15 22:06:03.722 | ERROR    | twscrape.models:_write_dump:418 - Failed to parse response of tweet, writing dump to /tmp/twscrape/twscrape_parse_error_2024-01-15_15-06-03_twwcy.txt
2024-01-15 22:06:09.789 | ERROR    | twscrape.models:_write_dump:418 - Failed to parse response of tweet, writing dump to /tmp/twscrape/twscrape_parse_error_2024-01-15_15-06-09_uosyo.txt
100%|██████████████████████████████████████████████████████████████████████████████| 365/365 [7:24:54<00:00, 73.14s/it]


In [ ]:
# import time
# df = pd.DataFrame()
# for end_point in tqdm.tqdm(end_points):
#     df_t = await get_data(end_point)
#     df = pd.concat([df,df_t])
#     time.sleep(np.random.randint(40))

In [12]:
df

,conversationId,coordinates,date,dict,hashtags,id,id_str,inReplyToTweetId,inReplyToUser,json,...,rawContent,replyCount,retweetCount,retweetedTweet,source,sourceLabel,sourceUrl,url,user,viewCount
0,1273042663131021313,None,2020-06-16 23:59:57+00:00,<bound method JSONTrait.dict of Tweet(id=12730...,"[bitcoin, stonks, shrutebucks]",1273042663131021313,1273042663131021313,None,None,<bound method JSONTrait.json of Tweet(id=12730...,...,Somebody needs to get @daveportnoy11 on a bitc...,0,0,None,"<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone,http://twitter.com/download/iphone,https://twitter.com/GabeCoin/status/1273042663...,"User(id=524569537, id_str='524569537', url='ht...",None
0,1270701608070868992,None,2020-06-16 23:59:45+00:00,<bound method JSONTrait.dict of Tweet(id=12730...,"[gochain, Bitcoin]",1273042613105352704,1273042613105352704,1271858895938392065,"UserRef(id=416961056, username='seven_yakuza71...",<bound method JSONTrait.json of Tweet(id=12730...,...,update for lonterm @go_chain #gochain #Bitcoin...,1,0,None,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,https://mobile.twitter.com,https://twitter.com/seven_yakuza71/status/1273...,"User(id=416961056, id_str='416961056', url='ht...",None
0,1273042517127200769,None,2020-06-16 23:59:22+00:00,<bound method JSONTrait.dict of Tweet(id=12730...,"[USD, EUR, CNY, GBP, RUB]",1273042517127200769,1273042517127200769,None,None,<bound method JSONTrait.json of Tweet(id=12730...,...,Bitcoin price index https://t.co/o7UcHJUhC6 #U...,0,0,None,"<a href=""http://www.worldcoinindex.com"" rel=""n...",WCI Trending,http://www.worldcoinindex.com,https://twitter.com/WorldCoinIndex/status/1273...,"User(id=2395138046, id_str='2395138046', url='...",None
0,1273042502195576835,None,2020-06-16 23:59:19+00:00,<bound method JSONTrait.dict of Tweet(id=12730...,"[cnbc, foxbusiness, business, money, entrepren...",1273042502195576835,1273042502195576835,None,None,<bound method JSONTrait.json of Tweet(id=12730...,...,"Nation files lawsuit against Benally, alleging...",0,1,None,"<a href=""http://www.musicnews360.com"" rel=""nof...",Music News 360 - Recommendation,http://www.musicnews360.com,https://twitter.com/musicnews360/status/127304...,"User(id=829150329543544832, id_str='8291503295...",None
0,1273042481412784128,None,2020-06-16 23:59:14+00:00,<bound method JSONTrait.dict of Tweet(id=12730...,[Bitcoin],1273042481412784128,1273042481412784128,None,None,<bound method JSONTrait.json of Tweet(id=12730...,...,"Running away may save you for the time being, ...",0,0,None,"<a href=""http://twitter.com/download/android"" ...",Twitter for Android,http://twitter.com/download/android,https://twitter.com/Cryptoplanet8/status/12730...,"User(id=1265654527094599680, id_str='126565452...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1477417977154875392,None,2022-01-01 23:56:40+00:00,<bound method JSONTrait.dict of Tweet(id=14774...,[],1477428591428263940,1477428591428263940,1477428269012115460,"UserRef(id=802542622480941056, username='Admir...",<bound method JSONTrait.json of Tweet(id=14774...,...,"@AdmiralEnte This is a good point, Bitcoin is ...",0,0,None,"<a href=""http://twitter.com/download/android"" ...",Twitter for Android,http://twitter.com/download/android,https://twitter.com/JosephPolitano/status/1477...,"User(id=3309908860, id_str='3309908860', url='...",None
0,1477417977154875392,None,2022-01-01 23:56:39+00:00,<bound method JSONTrait.dict of Tweet(id=14774...,[],1477428590174109696,1477428590174109696,1477420352754098176,"UserRef(id=3309908860, username='JosephPolitan...",<bound method JSONTrait.json of Tweet(id=14774...,...,@JosephPolitano @bhboehlert Ethereum is proof ...,0,0,None,"<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone,http://twitter.com/download/iphone,https://twitter.com/sensiblemoderat/status/147...,"User(id=392147989, id_str='392147989', url='ht...",None
0,147740531232808

In [ ]:
# for c in COLUMNS:
#     x = getattr(df_t, c)
#     print(c, x)
    
#         print("DEBUG len")
#         print(len(x))

In [13]:
print(df['date'].min())
print(df['date'].max())

2009-01-11 03:33:52+00:00
2022-01-01 23:59:54+00:00


In [14]:
df['d'] = pd.to_datetime(df['date'], utc=True).dt.date
df['date'] = pd.to_datetime(df['date'], utc=True).dt.tz_localize(None)

In [15]:
df.groupby('d')['date'].count()

d
2009-01-11     1
2013-12-19     1
2017-01-02     1
2017-10-14     1
2018-03-12     1
              ..
2021-12-24    56
2021-12-26    60
2021-12-28    56
2021-12-30    58
2022-01-01    60
Name: date, Length: 506, dtype: int64

In [16]:
pd.options.display.max_columns = 100

In [17]:
df

,conversationId,coordinates,date,dict,hashtags,id,id_str,inReplyToTweetId,inReplyToUser,json,lang,likeCount,links,media,mentionedUsers,parse,place,quoteCount,quotedTweet,rawContent,replyCount,retweetCount,retweetedTweet,source,sourceLabel,sourceUrl,url,user,viewCount,d
0,1273042663131021313,None,2020-06-16 23:59:57,<bound method JSONTrait.dict of Tweet(id=12730...,"[bitcoin, stonks, shrutebucks]",1273042663131021313,1273042663131021313,None,None,<bound method JSONTrait.json of Tweet(id=12730...,en,0,[],"Media(photos=[], videos=[], animated=[])","[UserRef(id=1107417216885157889, username='dav...",<function Tweet.parse at 0x00000169B7D75090>,None,0,None,Somebody needs to get @daveportnoy11 on a bitc...,0,0,None,"<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone,http://twitter.com/download/iphone,https://twitter.com/GabeCoin/status/1273042663...,"User(id=524569537, id_str='524569537', url='ht...",None,2020-06-16
0,1270701608070868992,None,2020-06-16 23:59:45,<bound method JSONTrait.dict of Tweet(id=12730...,"[gochain, Bitcoin]",1273042613105352704,1273042613105352704,1271858895938392065,"UserRef(id=416961056, username='seven_yakuza71...",<bound method JSONTrait.json of Tweet(id=12730...,en,3,[],Media(photos=[MediaPhoto(url='https://pbs.twim...,"[UserRef(id=949313959445766144, username='go_c...",<function Tweet.parse at 0x00000169B7D75090>,None,0,None,update for lonterm @go_chain #gochain #Bitcoin...,1,0,None,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,https://mobile.twitter.com,https://twitter.com/seven_yakuza71/status/1273...,"User(id=416961056, id_str='416961056', url='ht...",None,2020-06-16
0,1273042517127200769,None,2020-06-16 23:59:22,<bound method JSONTrait.dict of Tweet(id=12730...,"[USD, EUR, CNY, GBP, RUB]",1273042517127200769,1273042517127200769,None,None,<bound method JSONTrait.json of Tweet(id=12730...,en,0,[TextLink(url='https://www.worldcoinindex.com/...,Media(photos=[MediaPhoto(url='https://pbs.twim...,[],<function Tweet.parse at 0x00000169B7D75090>,None,0,None,Bitcoin price index https://t.co/o7UcHJUhC6 #U...,0,0,None,"<a href=""http://www.worldcoinindex.com"" rel=""n...",WCI Trending,http://www.worldcoinindex.com,https://twitter.com/WorldCoinIndex/status/1273...,"User(id=2395138046, id_str='2395138046', url='...",None,2020-06-16
0,1273042502195576835,None,2020-06-16 23:59:19,<bound method JSONTrait.dict of Tweet(id=12730...,"[cnbc, foxbusiness, business, money, entrepren...",1273042502195576835,1273042502195576835,None,None,<bound method JSONTrait.json of Tweet(id=12730...,en,2,[],Media(photos=[MediaPhoto(url='https://pbs.twim...,[],<function Tweet.parse at 0x00000169B7D75090>,None,0,None,"Nation files lawsuit against Benally, alleging...",0,1,None,"<a href=""http://www.musicnews360.com"" rel=""nof...",Music News 360 - Recommendation,http://www.musicnews360.com,https://twitter.com/musicnews360/status/127304...,"User(id=829150329543544832, id_str='8291503295...",None,2020-06-16
0,1273042481412784128,None,2020-06-16 23:59:14,<bound method JSONTrait.dict of Tweet(id=12730...,[Bitcoin],1273042481412784128,1273042481412784128,None,None,<bound method JSONTrait.json of Tweet(id=12730...,en,1,[],"Media(photos=[], videos=[], animated=[])",[],<function Tweet.parse at 0x00000169B7D75090>,None,0,None,"Running away may save you for the time being, ...",0,0,None,"<a href=""http://twitter.com/download/android"" ...",Twitter for Android,http://twitter.com/download/android,https://twitter.com/Cryptoplanet8/status/12730...,"User(id=1265654527094599680, id_str='126565452...",None,2020-06-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1477417977154875392,None,2022-01-01 23:56:40,<bound method JSONTrait.dict of Tweet(id=14774...,[],1477428591428263940,1477428591428263940,1477428269012115460,"UserRef(id=802542622480941056, username='Admir...",<bound method JSONTrait.json of Tweet(id=14774...,en,1,[],"Media

In [18]:
df['conversationId'].nunique()

20749

In [19]:
df.to_excel(f"./data/bitcoin_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}.xlsx", index=False)

In [144]:
a = pd.read_excel('./data/bitcoin_2023_2024.xlsx')

In [145]:
a

,conversationId,date,dict,id,id_str,inReplyToTweetId,inReplyToUser,json,lang,media,mentionedUsers,parse,rawContent,source,sourceLabel,sourceUrl,url,user,viewCount,d
0,1612962478677283072,2023-01-10 23:59:56,<bound method JSONTrait.dict of Tweet(id=16129...,1612962478677283072,1612962478677282819,NaN,"UserRef(id=2542046377, username='Netcoins', di...",<bound method JSONTrait.json of Tweet(id=16129...,en,"Media(photos=[], videos=[], animated=[])","UserRef(id=2542046377, username='Netcoins', di...",<function Tweet.parse at 0x000001DA8F25BC70>,@Netcoins @BiggDigitalCo closed my account aft...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,https://mobile.twitter.com,https://twitter.com/GAkrypto/status/1612962478...,"User(id=1469064819441647631, id_str='146906481...",71.0,2023-01-10
1,1612962478677283072,2023-01-10 23:59:56,<bound method JSONTrait.dict of Tweet(id=16129...,1612962478677283072,1612962478677282819,NaN,"UserRef(id=2542046377, username='Netcoins', di...",<bound method JSONTrait.json of Tweet(id=16129...,en,"Media(photos=[], videos=[], animated=[])","UserRef(id=1178807698042675200, username='Bigg...",<function Tweet.parse at 0x000001DA8F25BC70>,@Netcoins @BiggDigitalCo closed my account aft...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,https://mobile.twitter.com,https://twitter.com/GAkrypto/status/1612962478...,"User(id=1469064819441647631, id_str='146906481...",71.0,2023-01-10
2,1612875092328955904,2023-01-10 23:59:47,<bound method JSONTrait.dict of Tweet(id=16129...,1612962438592299008,1612962438592299012,1.612962e+18,"UserRef(id=128432873, username='Astropin', dis...",<bound method JSONTrait.json of Tweet(id=16129...,en,"Media(photos=[], videos=[], animated=[])","UserRef(id=128432873, username='Astropin', dis...",<function Tweet.parse at 0x000001DA8F25BC70>,"@Astropin Yes, one I do not regret.","<a href=""http://twitter.com/download/iphone"" r...",Twitter for iPhone,http://twitter.com/download/iphone,https://twitter.com/w_s_bitcoin/status/1612962...,"User(id=1381699484988637194, id_str='138169948...",20.0,2023-01-10
3,1612919662244666880,2023-01-10 23:59:46,<bound method JSONTrait.dict of Tweet(id=16129...,1612962435219987968,1612962435219988481,1.612920e+18,"UserRef(id=1510783080381964293, username='Amik...",<bound method JSONTrait.json of Tweet(id=16129...,sl,"Media(photos=[], videos=[], animated=[])","UserRef(id=1510783080381964293, username='Amik...",<function Tweet.parse at 0x000001DA8F25BC70>,@Amika_marksisto Tiu ago nur kaŭzas inflacion....,"<a href=""http://twitter.com/download/android"" ...",Twitter for Android,http://twitter.com/download/android,https://twitter.com/Cxapelo/status/16129624352...,"User(id=1152056306, id_str='1152056306', url='...",26.0,2023-01-10
4,1612935492542296064,2023-01-10 23:59:29,<bound method JSONTrait.dict of Tweet(id=16129...,1612962361471631104,1612962361471631360,1.612935e+18,"UserRef(id=2307692107, username='Blockstream',...",<bound method JSONTrait.json of Tweet(id=16129...,en,"Media(photos=[], videos=[], animated=[])","UserRef(id=2307692107, username='Blockstream',...",<function Tweet.parse at 0x000001DA8F25BC70>,@Blockstream In such long timescale you can be...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,https://mobile.twitter.com,https://twitter.com/Mnh_krpt/status/1612962361...,"User(id=1497028371930390529, id_str='149702837...",32.0,2023-01-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
669,1746054675106053888,2024-01-13 06:20:49,<bound method JSONTrait.dict of Tweet(id=17460...,1746054675106053888,1746054675106054302,NaN,NaN,<bound method JSONTrait.json of Tweet(id=17460...,vi,"Media(photos=[], videos=[], animated=[])","UserRef(id=1600770363671347200, username='tapc...",<function Tweet.parse at 0x000001DA8F25BC70>,Chủ tịch #SEC Hoa Kỳ từ chối bình luận cụ thể ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Twitter Web App,https://mobile.twitter.